# Logging Structure

* [logging — Logging facility for Python](https://docs.python.org/3/library/logging.html#module-logging)

> This page contains the API reference information. 
> * [Basic Tutorial](https://docs.python.org/3/howto/logging.html#logging-basic-tutorial)
> * [Advanced Tutorial](https://docs.python.org/3/howto/logging.html#logging-advanced-tutorial)
> * [Logging Cookbook](https://docs.python.org/3/howto/logging-cookbook.html#logging-cookbook)


## Components

1. **LogRecord** represents the vent to report.
2. **Loggers** expose the interface that application code directly uses.
3. **Handlers** send the log records (created by loggers) to the appropriate destination.
4. **Filters** provide a finer grained facility for determining which log records to output.
5. **Formatters** specify the layout of log records in the final output.


# [LogRecord](https://docs.python.org/3/library/logging.html#logging.LogRecord)

> LogRecord instances are created automatically by the Logger every time something is logged, and can be created manually via makeLogRecord() (for example, from a pickled event received over the wire).
> * name – The name of the logger used to log the event represented by this LogRecord. Note that this name will always have this value, even though it may be emitted by a handler attached to a different (ancestor) logger.
> * level – The numeric level of the logging event (one of DEBUG, INFO etc.) Note that this is converted to two attributes of the LogRecord: levelno for the numeric value and levelname for the corresponding level name.
> * pathname – The full pathname of the source file where the logging call was made.
> * lineno – The line number in the source file where the logging call was made.
> * msg – The event description message, possibly a format string with placeholders for variable data.
> * args – Variable data to merge into the msg argument to obtain the event description.
> * exc_info – An exception tuple with the current exception information, or None if no exception information is available.
> * func – The name of the function or method from which the logging call was invoked.
> * sinfo – A text string representing stack information from the base of the stack in the current thread, up to the logging call.

# [Logger](https://docs.python.org/3/library/logging.html#logger-objects)

```logging.getLogger(__name__)``` with the same name always returns a reference to the same Logger object.

### name

**name** should **reflect module hierarchy** with period-separated e.g. foo.bar.baz, because loggers with names of foo.bar, foo.bar.baz, and foo.bam are all **descendants** of foo.

### [propagate](https://docs.python.org/3/library/logging.html#logging.Logger.propagate)

> If this attribute **evaluates to true, events logged to this logger will be passed to the handlers** of higher level (ancestor) loggers, in addition to any handlers attached to this logger. Messages are passed directly to the ancestor loggers’ handlers - neither the level nor filters of the ancestor loggers in question are considered.


### [handle](https://docs.python.org/3/library/logging.html#logging.Logger.handle)

> Pass log events to all handlers associated with this logger **and its ancestors**

### [getChild(suffix)](https://docs.python.org/3/library/logging.html#logging.Logger.getChild)

> Returns a logger which is a descendant to this logger

### [addFilter](https://docs.python.org/3/library/logging.html#logging.Logger.addFilter)

> Adds the specified filter filter to this logger.

# [Handler](https://docs.python.org/3/library/logging.handlers.html#module-logging.handlers)

> For a list of handlers included as standard, see [logging.handlers](https://docs.python.org/3/library/logging.handlers.html#module-logging.handlers).

# [Formatter](https://docs.python.org/3/library/logging.html#formatter-objects)

> Responsible for **converting a LogRecord** to what can be interpreted by either a human or an external system. 


# [Filter](https://docs.python.org/3/library/logging.html#filter-objects)

Controls how the logging events flow through the logger hierarchy. A filter initialized with ```A.B``` allows events loggers ```A.B```, ```A.B.C```, ```A.B.C.D```, ```A.B.D```, but not ```A.BB```, ```B.A.B``` etc. If initialized with the empty string, all events are passed.

> Note that filters attached to handlers are consulted before an event is emitted by the handler, whereas filters attached to loggers are consulted whenever an event is logged (using debug(), info(), etc.), before sending an event to handlers. This means that events which have been generated by descendant loggers will not be filtered by a logger’s filter setting, unless the filter has also been applied to those descendant loggers.

---
# [Thread safety](https://docs.python.org/3/library/logging.html#thread-safety)

> The logging module is intended to be thread-safe without any special work needing to be done by its clients. 

---
# Standard practice

* [Advanced Logging Tutoria](https://docs.python.org/3/howto/logging.html#logging-advanced-tutorial)

## logger names reflect the package/module hierarchy
                             
A good convention to use when naming loggers is to use a module-level logger, in each module which uses logging, named as follows:

```
logger = logging.getLogger(__name__)
```

This means that logger names track the package/module hierarchy, and it’s intuitively obvious where events are logged just from the logger name.

---
# basicConfig

* [Basic Logging Tutorial](https://docs.python.org/3/howto/logging.html)


In [ ]:
logging.basicConfig(filename='example.log', filemode='w', level=logging.DEBUG)

---
# Control logging

* [Should be able to turn off logging #3050](https://github.com/huggingface/transformers/issues/3050#issuecomment-682167272)

Usage:

1. override all module-specific loggers to a desired level (except whatever got logged during modules importing)
```
import everything, you, need
import logging
set_global_logging_level(logging.ERROR)
```

2. In case of transformers you most likely need to call it as:
```
import transformers, torch, ...
import logging
set_global_logging_level(logging.ERROR, ["transformers", "nlp", "torch", "tensorflow", "tensorboard", "wandb"])
```

To disable logging globally - place at the beginning of the script
```
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
# logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere
```

In [1]:
import logging
import re
def set_global_logging_level(level=logging.ERROR, prefices=[""]):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        - level: desired level. e.g. logging.INFO. Optional. Default is logging.ERROR
        - prefices: list of one or more str prefices to match (e.g. ["transformers", "torch"]). Optional.
          Default is `[""]` to match all active loggers.
          The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{ "|".join(prefices) })')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)

---
# Optimization

Formatting of message arguments is deferred until it cannot be avoided.Call the isEnabledFor() method.

* [Logging - Optimization](https://docs.python.org/3/howto/logging.html#optimization)

```
if logger.isEnabledFor(logging.DEBUG):
    logger.debug(
        'Message with %s, %s', expensive_func1(), expensive_func2()
    )
```

The calls to expensive_func1() and expensive_func2() are never made if the logger’s threshold is set above DEBUG.